# Check macroeconomic index

In [1]:
from typing import List
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

## Unemployment Rate

import data

In [2]:
file = Path("../../data/raw/macro/BB.xlsx")
df = pd.read_excel(
    file,
    sheet_name=1,
    skiprows=[0, 1, 3, 4, 5],
    index_col=0,
)
df.index.name = "date" # Rename index
df.index = pd.to_datetime(df.index) # Convert type of index to datetime
# df.drop(columns=["EHUPUS Index"], inplace=True) # Drop non-seasonal-adjusted unemployment rate
df = df[df.index.is_month_end] # Only month end
df.dropna(inplace=True) # Drop NaN (just in case) 

display(df.tail())

,USURTOT Index,EHUPUS Index,UEMPREDU Index,UEMPRCON Index,UEMPRMIN Index,UEMPRMAN Index,UEMPRFNC Index,UEMPRLES Index,UEMPRSLF Index,UEMPRAGR Index,UEMPRNAG Index,UEMPRTRN Index,UEMPRTIL Index,UEMPRBUS Index,UEMPRDUR Index,UEMPRGOV Index,UEMPRINF Index,UEMPRNON Index,UEMPRSVC Index
date,,,,,,,,,,,,,,,,,,,
2022-03-31,3.6,3.80,2.5,6.0,2.6,3.1,2.0,5.9,3.2,7.0,3.9,5.1,4.6,4.0,3.5,1.5,2.3,2.5,3.2
2022-04-30,3.6,3.60,2.2,4.6,3.4,3.2,1.9,4.8,3.7,4.3,3.3,3.4,4.2,3.0,3.3,1.5,2.7,3.0,3.3
2022-05-31,3.6,3.60,2.5,3.8,4.1,2.8,1.9,5.1,2.6,3.1,3.3,4.1,4.2,3.1,2.7,1.7,2.9,2.9,3.1
2022-06-30,3.6,3.60,3.3,3.7,1.6,3.0,2.1,5.2,2.7,4.4,3.5,3.8,4.1,3.4,2.9,3.1,3.1,3.2,2.5
2022-07-31,3.5,3.57,3.2,3.5,0.8,3.2,1.6,4.8,2.3,4.7,3.4,3.7,4.1,3.3,2.9,3.8,2.4,3.7,2.8


import description

In [3]:
file = Path("../../data/raw/macro/BB.xlsx")
df_description = pd.read_excel(
    file,
    sheet_name=1,
    index_col=0,
    header=1
).iloc[[0], :]

df_description.index.name = None # Rename Index
df_description = df_description.T.reset_index(names=["Description"]) # Transpose and Rename columns
df_description.set_index("Ticker", inplace=True) # Set Index

display(df_description.head())

,Description
Ticker,
USURTOT Index,Total_SA
EHUPUS Index,Total
UEMPREDU Index,Education & Health Searvices
UEMPRCON Index,Construction Industry
UEMPRMIN Index,Mining/Quarrying/Oil & Gas Extraction


Plot

In [4]:
def seasonal_plot(df:pd.DataFrame, df_description:pd.DataFrame=None, colors:List[str]=px.colors.qualitative.Dark24):
    traces = [] # trace list
    for i, ticker in enumerate(df.columns):
        name = df_description.loc[ticker].values[0] if df_description is not None else ticker # label name
        trace = go.Scatter(
            x = df.index,
            y = df[ticker],
            name = name,
            marker_color = colors[i%len(colors)],
        )
        traces.append(trace)
    layout = go.Layout(
        width=1400,
        height=800,
        font_size=15,
        legend={"x":0.99, "y":1.1, "xanchor":"right", "yanchor":"bottom","orientation":"h"},
        xaxis={"title":"Date"},
        yaxis={"title":"Unemployment Rate [%]", "side":"left"},
        hovermode="x",
        template="plotly_dark",
    )
    fig = go.Figure(data=traces, layout=layout)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=[
                {
                    "count":i, "label":f"last {i} year", "step":"year",
                } for i in [1, 3, 5, 10]
            ],
            bgcolor="#2F301F"
        )
    )
    return fig

In [5]:
fig = seasonal_plot(df, df_description=df_description)
fig.show()

Data should be seasonal adjasted.

# Unemployment Forecast

import data

In [6]:
file = Path("../../data/raw/macro/BB.xlsx")
df = pd.read_excel(
    file,
    sheet_name=2,
    skiprows=[0, 1, 3, 4, 5],
    index_col=0,
)
df.index.name = "date" # Rename index
df.index = pd.to_datetime(df.index) # Convert type of index to datetime
# df.drop(columns=["EHUPUS Index"], inplace=True) # Drop non-seasonal-adjusted unemployment rate
df = df[df.index.is_month_end] # Only month end
df.dropna(inplace=True, axis=1, how="all") # Drop NaN
df.dropna(inplace=True, axis=0, how="all") # Drop NaN

display(df.tail())

,ECUPUS 10 Index,ECUPUS 11 Index,ECUPUS 12 Index,ECUPUS 13 Index,ECUPUS 14 Index,ECUPUS 15 Index,ECUPUS 16 Index,ECUPUS 17 Index,ECUPUS 18 Index,ECUPUS 19 Index,ECUPUS 20 Index,ECUPUS 21 Index,ECUPUS 22 Index,ECUPUS 23 Index,ECUPUS 24 Index
date,,,,,,,,,,,,,,,
2022-02-28,9.6,9.0,8.1,7.4,6.2,5.3,4.9,4.4,3.9,3.7,8.1,5.4,3.60,3.6,3.80
2022-03-31,9.6,9.0,8.1,7.4,6.2,5.3,4.9,4.4,3.9,3.7,8.1,5.4,3.60,3.6,3.85
2022-04-30,9.6,9.0,8.1,7.4,6.2,5.3,4.9,4.4,3.9,3.7,8.1,5.4,3.65,4.0,4.20
2022-05-31,9.6,9.0,8.1,7.4,6.2,5.3,4.9,4.4,3.9,3.7,8.1,5.4,3.70,4.1,4.30
2022-06-30,9.6,9.0,8.1,7.4,6.2,5.3,4.9,4.4,3.9,3.7,8.1,5.4,3.70,4.3,4.60


In [7]:
fig = seasonal_plot(df)
fig.show()

## IC, CC

import data

In [8]:
dfs = [] # data-frame list

for factor in ["ICSA", "CCSA"]:
    dir = Path("../../data/raw/macro")
    df = pd.read_csv(dir/ f"{factor}.csv")
    df.set_index("date", inplace=True)
    df.index = pd.to_datetime(df.index) # Convert type of index to datetime
    dfs.append(df)
    
df = pd.concat(dfs, axis=1)
display(df.tail())

,ICSA,CCSA
date,,
2022-09-10,208000.0,1376000.0
2022-09-17,209000.0,1346000.0
2022-09-24,190000.0,1365000.0
2022-10-01,219000.0,1368000.0
2022-10-08,228000.0,NaN


In [9]:
fig = seasonal_plot(df)
fig.show()

## SP500, NASDAQ, DOW

In [10]:
dfs = [] # data-frame list

for factor in ["SP500", "NASDAQCOM", "DJIA"]:
    dir = Path("../../data/raw/macro")
    df = pd.read_csv(dir/ f"{factor}.csv")
    df.set_index("date", inplace=True)
    df.index = pd.to_datetime(df.index) # Convert type of index to datetime
    dfs.append(df)
    
df = pd.concat(dfs, axis=1)
display(df.tail())

,SP500,NASDAQCOM,DJIA
date,,,
2022-10-10,3612.39,10542.10,29202.88
2022-10-11,3588.84,10426.19,29239.19
2022-10-12,3577.03,10417.10,29210.85
2022-10-13,3669.91,10649.15,30038.72
2022-10-14,3583.07,10321.39,29634.83


In [11]:
fig = seasonal_plot(df)
fig.show()